In [ ]:
%pip install -q xdai #installer modulen

<img src="gpr_vis_9.png" width = 100%>

# Med kunstig intelligens som labpartner

<a href="https://hazelbright.github.io/xdai">XDAI</a> og dette dokumentet er utviklet av Audun Skau Hansen, Stian Kogler, Hanne Røberg-Larsen, Steven H. R. Wilson, Elias Dalan, Ayla S. Coder, Hanan Gharayba og Maren H.S. Johnsen ved Universitetet i Oslo, våren 2023.

---

**Hvordan får vi mest mulig informasjon ut av et eksperiment med færrest mulig målinger? La oss se på en mulig fremgangsmåte som vil kunne hjelpe deg å løse dette problemet.**

Moderne maskinlærings(ML)-teknikker er gjerne basert på store **datasett**, bestående av "inn-data" i form av **målinger** (innen ML typisk kalt "features") med tilhørende "ut-data" i form av **resultater** (innen ML gjerne kalt "labels" eller "targets"). En ML-teknikk som kalles "supervised learning" har som mål å trene en **modell** så den kan gjøre **prediksjoner** i samsvar med datasettet. Du har kanskje vært borti dette før gjennom lineær regresjon, hvor modellen er et polynom.

<p></p>
<div>
    <img src="regression_model_4.png" width = 60%>
    <center>Maskinlæring: et datasett med målinger og resultater brukes for å lære opp en modell på datamaskinen.</center>
</div>
<p></p>
<p></p>



## Bayesiansk optimering

Eksperimenter kan imidlertid være tids- og arbeidskrevende, og vi har ikke alltid tilgang på store datasett. I tillegg kan det være en utfordring at vi ikke har forutsetning for å anta noe om den matematiske formen på resultatene. I slike tilfeller passer det å benytte noe som kalles **Gaussiske prosesser** (GP) som modell. Om vi benytter GP for å predikere et maksimum (topp- eller bunnpunkt) i målingene kalles dette for **Bayesiansk optimering** (BO). 

En fordel med denne metoden er at den kan ta måleusikkerhet med i betraktningen, noe som gjør den spesielt godt egnet til vårt formål.

Om du ønsker en mer detaljert introduksjon til Bayesiansk optimering kan vi anbefale følgende ressurser:

## Mål for denne notebooken

Denne notebooken vil lære deg å bruke den studentutviklede Python-modulen <a href="">XDAI</a> som støtte i ditt eksperimentelle arbeid. Notebooken er delt inn i fire deler:
1. **Eksperimentell design:**  planlegge målingene dine.
2. **Regresjon:** Tilpasse en Gaussisk prosess regressor (GPR) til datasettet ditt.
3. **Prediksjon:** Predikere en måling for optimalt resultat. (Bayesiansk optimering)
4. **Fortolkning** Visualisering av resultater.

Målet er å gjøre deg i stand til å finne optimale målepunkter for et bredt utvalg problemstillinger. Husk imidlertid at maskinlæringsmetoder ikke kan bli bedre enn de datasettene vi trener algoritmen på, så presise målinger og kritisk refleksjon rundt resultatene er avgjørende for å oppnå gode resultater.

<div class="alert alert-block alert-info">
    <h2> Diskusjonsoppgaver</h2>
<ul>    
 <li>Gi 3 eksempler på "inn-data" og "ut-data" (fra eksperiment, statistikk eller hverdagen) </li>
<li>Hvordan tolker du det å "predikere et maksimum i målingene"? </li>
<li>Hva innebærer det å optimere noe? Kan du gi noen eksempler på optimering? (hverdagslig eller vitenskapelig)</li>
<li>Kan du tenke deg tilfeller hvor du kan anta noe om formen på den matematiske formen på resultatene?</li>
<li>Kan du tenke deg tilfeller hvor du ikke kan anta noe om denne formen?</li>
<li>Hva legger du i det å "lære opp en modell på datamaskinen"? Hvordan tror du dette gjøres i praksis?</li>
    </ul>
</div>


## 1. **Eksperimentell design:** å planlegge målinger

Forestill deg at du vil finne en optimal sammensetning av målevariablene $\mathbf{x}_*$ som inngår i et eksperiment (for eksempel temperatur, trykk, konsentrasjoner, varighet og lignende). Hver måling er tidkrevende, så du ønsker å gjøre så få målinger som mulig. 

Du planlegger derfor et begrenset antall målinger i et område hvor du tror at den optimale sammensetningen befinner seg. Fra disse målingene vil du estimere en mer eksakt plassering av optimum. 

Det å planlegge disse målingene kalles **eksperimentell design**. Vi kommer i det følgende til å avgrense måleområdet med nedre og øvre terskelverdier i målevariablene. 

I en dimensjon er dette enkelt. Forestill deg for eksempel at du skal finne en optimal temperatur for veksten av en bakteriekultur. Du antar at bakteriene trives best ved romtemperatur, så du velger et måleintervall mellom 10 og 30 grader Celsius. Deretter gjør du målinger av vekstraten for 10 grader, 30 grader og kanskje for 20 grader (i midten av intervallet ditt). 

I det følgende bruker vi <a href="">XDAI</a> til å sette opp 1 og 2 dimensjoner. Du skal selve sette opp 3 dimensjoner.


In [1]:
import xdai

limits = [[10,30]]

grid_1d = xdai.designer.doe_grid( limits )

xdai.designer.html_table( grid_1d ) # display 1d table nicely formatted in noteobok

,20.0
,20.0
,20.0
,10.0
,30.0


In [20]:
limits = [[10,30], [0,1]]

grid_1d = xdai.designer.doe_grid( limits )

xdai.designer.html_table( grid_1d ) # display 2d table nicely formatted in noteobok

,20.0,0.5
,20.0,0.5
,20.0,0.5
,10.0,0.0
,30.0,0.0
,10.0,1.0
,30.0,1.0


In [21]:
limits = [ ... ]

grid_1d = xdai.designer.doe_grid( limits )

xdai.designer.html_table( grid_1d ) # display 1d table nicely formatted in noteobok

ValueError: x and y arrays must be equal in length along interpolation axis.

### Box-Behnken design

Det finnes flere ulike teknikker for å designe et eksperiment (se for eksempel <a href="https://www.itl.nist.gov/div898/handbook/pri/section5/pri5.htm">her</a>). Vi skal benytte det som kalles et Box-Behnken design []. Du setter det opp i XDAI på følgende måte:

In [15]:
import xdai 
import numpy as np

In [32]:
import numpy as np

limits = [ [0,7],[0 , 2.5]]

bb_grid = xdai.designer.doe_grid(limits, design = 2) # design=2 gir et Box-Behnken design
bb_grid

array([[3.5 , 1.25],
       [3.5 , 1.25],
       [3.5 , 1.25],
       [3.5 , 0.  ],
       [0.  , 1.25],
       [7.  , 1.25],
       [3.5 , 2.5 ]])

Du kan nå gjøre målingene som er vist i tabellen over. Om du bare vil leke deg litt med koden 

## Prediksjon

Å gjøre en prediksjon betyr å forutsi noe ukjent basert på det vi allerede vet. For å gjøre dette vil vi trene en 

In [8]:
målinger = doe_grid([ [-1,1], [-1,1], [-1,1]], design = 2)

html_table(målinger)

,1.0,0.0,0.0,0.0
,2.0,0.0,0.0,0.0
,3.0,0.0,0.0,0.0
,4.0,-1.0,-1.0,0.0
,5.0,0.0,-1.0,-1.0
,6.0,0.0,-1.0,1.0
,7.0,1.0,-1.0,0.0
,8.0,-1.0,0.0,-1.0
,9.0,-1.0,0.0,1.0
,10.0,1.0,0.0,-1.0
,11.0,1.0,0.0,1.0


In [ ]:
visualize_doe_grid()

## 2. Regresjon


